<a href="https://colab.research.google.com/github/ChaconLima/mestrado/blob/introdu%C3%A7%C3%A3o-a-meta-heuristica/Projeto_3_Introdu%C3%A7%C3%A3o_a_Meta_Heuristicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [350]:
#################################################################################################
# library Simulated Annealing - Copyright 2023 Mateus Chacon

# Este programa é um software livre, você pode redistribuí-lo e/ou modificá-lo
# sob os termos da Licença Pública Geral GNU como publicada pela Fundação do Software Livre (FSF),
# na versão 3 da Licença, ou (a seu critério) qualquer versão posterior.

# Este programa é distribuído na esperança de que possa ser útil, mas SEM NENHUMA GARANTIA,
# e sem uma garantia implícita de ADEQUAÇÃO a qualquer MERCADO ou APLICAÇÃO EM PARTICULAR.

# Veja a Licença Pública Geral GNU para mais detalhes
#################################################################################################

!pip install haversine

Buscando dados das Localizações

In [108]:
import pandas as pd
import io
import requests

class GetLocation:
  def __init__(self):
    url = 'https://github.com/kelvins/Municipios-Brasileiros/blob/master/csv/municipios.csv'
    # Fazer a requisição HTTP e obter o conteúdo
    response = requests.get(url)
    s = response.content

    # Criar um objeto StringIO a partir do conteúdo obtido
    file_name = io.StringIO(s.decode('utf-8'))

    # Ler o arquivo CSV utilizando o pandas
    df_geodata = pd.read_html(file_name)
    df_geodata = df_geodata[0]
    df_geodata.columns

    # Remover a coluna 'Unnamed: 0'
    df_geodata.drop(columns=['Unnamed: 0'], inplace=True)

    self.geoData = df_geodata

  def getGeoData(self):
    d = self.geoData
    data = {}
    for ad in range(len(d)):
      point = {
          'index': d.values[ad][0],
          'codigo_ibge':d.values[ad][1],
          'nome':d.values[ad][2],
          'latitude':d.values[ad][3],
          'longitude':d.values[ad][4]
      }
      data[ad] = point
    return data

  def getGeoDataByState(self, codState, limit):
    d = self.geoData[self.geoData['codigo_uf'] == codState].copy().reset_index()
    data = {}
    for ad in range(limit):
      point = {
          'index': d.values[ad][0],
          'codigo_ibge':d.values[ad][1],
          'nome':d.values[ad][2],
          'latitude':d.values[ad][3],
          'longitude':d.values[ad][4]
      }
      data[ad] = point
    return data


Classe de Gerenciamento do mapa

In [158]:
import folium

class Map:
  def __init__(self, data = {}):
    self.data = data
    self.map = 0

  def getMap(self):
    return self.map

  def initMap(self):
    coords_brasil = [-16.089461610967666, -55.910299337040286]
    coords_sp = [-23.5509378979565, -46.6398829009542]
    self.map = folium.Map(location= coords_sp, zoom_start=7, tiles="cartodbpositron")

  def createRoute(self, df=[]):
    n_cities = len(df)

    for i in range(len(df)):
      start_point = [self.data[df[i]]['latitude'],self.data[df[i]]['longitude']]
      if (i < (n_cities-1)):
        end_point  = [self.data[df[i+1]]['latitude'],self.data[df[i+1]]['longitude']]
      else:
        end_point  = [self.data[df[0]]['latitude'],self.data[df[0]]['longitude']]

      route = [start_point, end_point]
      folium.PolyLine(route,
                      weight=8,
                      color='red',
                      opacity=0.6).add_to(self.map)

  def drawCities(self,df=[]):
    #For each city draw CircleMarker point in the Map
    for i in range(len(df)):
        folium.CircleMarker(
            location=[self.data[df[i]]['latitude'], self.data[df[i]]['longitude']],
            radius = 6,
            popup=str(i+1)+': '+str(self.data[df[i]]['nome']),
            fill=True, # Set fill to True
            fill_color='lightblue',
            color='grey',
            fill_opacity=0.7
        ).add_to(self.map)

  def renderMap(self, df=[]):
    self.initMap()
    self.drawCities(df)
    self.createRoute(df)
    return self.map

Classe de Gerenciamento da Matriz distancia

In [256]:
import haversine as hs

class Direction():

  def __init__(self, data={}):
    self.directions = {}
    self.data = data
    self.genereteMD()

  def getKey(self, index_origin, index_destination):
    return str(index_origin)+':'+str(index_destination)

  def getMDist(self):
    return self.directions

  def getDistance(self, index_origin, index_destination):
    return self.directions[self.getKey(index_origin,index_destination)]['distance']

  def calcDistence(self, df):
    distance = 0
    for i in range(len(df)-1):
      origin = df[i]
      destination = df[i+1]
      distance += self.getDistance(origin, destination)
    return distance + self.getDistance(df[len(df)-1], df[0])


  def getRoute(self, origin, destination):
    distance  = hs.haversine(origin, destination)
    start_point = origin
    end_point   = destination
    routes = [origin, destination]
    out = {'route':routes,
           'start_point':start_point,
           'end_point':end_point,
           'distance':distance
          }
    return out

  def genereteMD(self):
    keys = list(self.data.keys())
    for i in range(len(keys)):
      for j in range(len(keys)):
        if(i!=j):
          origin = [self.data[keys[i]]['latitude'],self.data[keys[i]]['longitude']]
          destination = [self.data[keys[j]]['latitude'],self.data[keys[j]]['longitude']]
          self.directions[self.getKey(i,j)] = self.getRoute(origin,destination)


Classe que executa o algoritmo do visinho mais próximo

In [257]:
import copy

class Answer:
  def __init__(self, answer=[],eval=0):
    self.tour = copy.copy(answer)
    self.distance = copy.copy(eval)

  def getTour(self):
    return self.tour
  def getDistance(self):
    return self.distance


In [258]:
import math

class SolverTspNearest:
  def __init__(self, data, direction):
    self.data = data
    self.direction = direction

  def execute(self):
    num_cities = len(list(self.data))
    visited = [False] * num_cities
    tour = []
    total_distance = 0

    # Start at the first city
    current_city = 0
    tour.append(current_city)
    visited[current_city] = True


    # Repeat until all cities have been visited
    while len(tour) < num_cities:
        nearest_city = None
        nearest_distance = math.inf

        # Find the nearest unvisited city
        for city in range(num_cities):
            if not visited[city]:
                distance = self.direction.getDistance(current_city,city)
                if distance < nearest_distance:
                    nearest_city = city
                    nearest_distance = distance

        # Move to the nearest city
        current_city = nearest_city
        tour.append(current_city)
        visited[current_city] = True
        total_distance += nearest_distance

    # Complete the tour by returning to the starting city
    total_distance += self.direction.getDistance(current_city,0)
    return Answer(tour,total_distance)

In [342]:
from math import exp
import random
import copy

class SimulatedAnnealing:

  def __init__(self,data, direction, numberIterations=100, temp=1, seed = 1):
    self.scores=[]
    random.seed(seed)
    self.anwser=Answer
    self.numberIterations = numberIterations
    self.temp = temp
    self.data = data
    self.direction = direction

  def flip(self, route, cutPointer):
    soluction = route[:cutPointer]
    return soluction+route[cutPointer:][::-1]

  def anwserInitial(self):
    return SolverTspNearest(self.data,self.direction).execute()

  def generateAnwserCandidate(self, curr):
    sol = curr.getTour()
    n = len(sol)
    p = random.randint(0, n)
    sol2 = self.flip(sol,p)
    p = random.randint(0, n)
    sol2 = self.flip(sol2,p)
    return Answer(sol2, self.direction.calcDistence(sol2))

  def execute(self):
    best = self.anwserInitial()
    curr = copy.copy(best)
    self.scores.append(best.getDistance())

    for i in range(self.numberIterations):
      candidate = self.generateAnwserCandidate(curr)
      diff = candidate.getDistance() - curr.getDistance()
      if diff < 0:
        diff_global = candidate.getDistance() - best.getDistance()
        if diff_global < 0:
          best = copy.copy(candidate)
        curr = copy.copy(candidate)
        self.scores.append(curr.getDistance())
      else:
        t = self.temp/(i+1)
        prob = exp(-diff/t)
        if random.random() < prob:
          curr  = copy.copy(candidate)

    self.anwser = best
    return best


In [351]:
from datetime import datetime

data = GetLocation()
dataSP = data.getGeoDataByState(35, 300)
direction = Direction(dataSP)

seed = datetime.now().timestamp()
# seed = 1
answer = SimulatedAnnealing(
    data=dataSP,
    direction=direction,
    numberIterations=10000,
    temp=1000,
    seed=seed).execute()

print("Tour:", answer.getTour())
print("Total distance:",answer.getDistance())

map = Map(dataSP)
map.renderMap(answer.getTour())



Tour: [0, 240, 198, 196, 199, 293, 16, 192, 191, 181, 161, 94, 67, 7, 84, 41, 230, 59, 264, 82, 58, 253, 39, 220, 83, 262, 226, 194, 156, 115, 179, 42, 252, 248, 1, 292, 52, 280, 54, 244, 201, 128, 224, 166, 127, 126, 167, 105, 68, 277, 135, 62, 200, 246, 203, 276, 231, 33, 93, 289, 147, 187, 272, 98, 155, 3, 172, 299, 151, 55, 35, 40, 149, 97, 113, 210, 27, 110, 96, 284, 236, 120, 71, 213, 150, 153, 214, 44, 182, 266, 74, 209, 279, 232, 290, 86, 79, 47, 287, 269, 21, 4, 260, 174, 2, 123, 273, 124, 106, 11, 66, 89, 286, 146, 162, 63, 278, 212, 215, 19, 50, 57, 88, 92, 193, 184, 195, 48, 216, 154, 283, 175, 180, 206, 239, 158, 45, 28, 235, 265, 125, 23, 204, 69, 208, 31, 73, 140, 197, 87, 134, 190, 72, 12, 38, 221, 65, 291, 81, 217, 17, 173, 49, 53, 207, 99, 144, 20, 14, 122, 228, 10, 205, 136, 36, 223, 152, 22, 142, 271, 90, 159, 75, 76, 288, 157, 25, 6, 132, 243, 247, 139, 37, 137, 171, 43, 143, 218, 282, 108, 219, 18, 119, 165, 237, 275, 295, 109, 188, 189, 101, 103, 121, 64, 285, 25